I'm scrapping everything else I worked on to try and get a "mini" model up with a small set of the data. I've been struggling to get my head around how to set up the data but I think I've got a bead on something that will work.

In [3]:
import pandas as pd
import numpy as np

In [4]:
reviews = pd.read_csv('data/reviews_step3_output.csv')
reviews.head()

,Unnamed: 0,review_id,user_id,book_id,rating,review_text,year
0,0,66b2ba840f9bd36d6d27f46136fe4772,dc3763cdb9b2cae805882878eebb6a32,18471619,3,Sherlock Holmes and the Vampires of London \n ...,2013
1,1,72f1229aba5a88f9e72f0dcdc007dd22,bafc2d50014200cda7cb2b6acd60cd73,6315584,4,"I've never really liked Spider-Man. I am, howe...",2016
2,2,a75309355f8662caaa5e2c92ab693d3f,bafc2d50014200cda7cb2b6acd60cd73,29847729,4,"A very quick introduction, this is coming out ...",2016
3,3,c3cc5a3e1d6b6c9cf1c044f306c8e752,bafc2d50014200cda7cb2b6acd60cd73,18454118,5,I've been waiting so long for this. I first st...,2014
4,4,cc444be37ab0a42bfb4dd818cb5edd10,bafc2d50014200cda7cb2b6acd60cd73,2239435,4,The only thing more entertaining than this boo...,2013


In [5]:
# Trying this with the full data set keeps leading to error - too much data for the pivot function
small_set = reviews.iloc[:100]
ratings_matrix = pd.pivot_table(small_set, values='rating',index='user_id', columns='book_id',
                               fill_value=0)

In [6]:
ratings_matrix = ratings_matrix.loc[:,~ratings_matrix.columns.duplicated()]
display(ratings_matrix.shape)
display(ratings_matrix.head())

(28, 100)

book_id,9525,15195,15196,37261,38381,59708,62953,69735,95431,96358,...,29847729,30289606,30989347,31139997,31279606,31387205,31390214,32021495,32989782,34235800
user_id,,,,,,,,,,,,,,,,,,,,,
0d8d07544717e84149df654caae803d0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0ef32090550901ead25cb0ea21c4d36b,0,0,0,0,4,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18759758c7cf75975cff3fbb1363a73f,0,4,0,0,0,3,0,0,5,0,...,0,0,0,3,4,0,0,0,0,0
1afe8b35c5e568e95bc17e5b5cdbfd1b,0,0,5,0,0,0,0,0,0,0,...,0,5,0,0,0,4,5,4,3,0
2f6af21d14c83a5df6cdcef5e6af0b3e,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
from sklearn.metrics.pairwise import pairwise_distances

user_similarity = pairwise_distances(ratings_matrix, metric='cosine')

item_similarity = pairwise_distances(ratings_matrix.T, metric='cosine')

In [8]:
user_similarity[5]

array([1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [9]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff)/np.array([
            np.abs(similarity).sum(axis=1)
        ]).T
        
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([
            np.abs(similarity).sum(axis=1)
        ])
        
    return pred

In [10]:
user_prediction = predict(ratings_matrix, user_similarity, type='user')

In [11]:
display(type(user_prediction))
display(user_prediction.shape)

numpy.ndarray

(28, 100)

In [12]:
display(pd.DataFrame(user_prediction))

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.105556,0.105556,0.142593,0.142593,0.105556,0.068519,0.068519,0.105556,0.142593,0.142593,...,0.105556,0.142593,0.068519,0.068519,0.105556,0.105556,0.142593,0.105556,0.068519,0.142593
1,0.209259,0.209259,0.246296,0.246296,0.061111,0.172222,0.061111,0.209259,0.246296,0.246296,...,0.209259,0.246296,0.172222,0.172222,0.209259,0.209259,0.246296,0.209259,0.172222,0.246296
2,0.427037,0.278889,0.464074,0.464074,0.427037,0.278889,0.390000,0.427037,0.278889,0.464074,...,0.427037,0.464074,0.390000,0.278889,0.278889,0.427037,0.464074,0.427037,0.390000,0.464074
3,0.468519,0.468519,0.320370,0.505556,0.468519,0.431481,0.431481,0.468519,0.505556,0.505556,...,0.468519,0.320370,0.431481,0.431481,0.468519,0.320370,0.320370,0.320370,0.320370,0.505556
4,0.095185,0.095185,0.132222,0.132222,0.095185,0.058148,0.058148,0.095185,0.132222,0.132222,...,0.095185,0.132222,0.058148,0.058148,0.095185,0.095185,0.132222,0.095185,0.058148,0.132222
5,0.053704,0.053704,0.090741,0.090741,0.053704,0.016667,0.016667,0.053704,0.090741,0.090741,...,0.053704,0.090741,0.016667,0.016667,0.053704,0.053704,0.090741,0.053704,0.016667,0.090741
6,0.105556,0.105556,0.142593,0.142593,0.105556,0.068519,0.068519,0.105556,0.142593,0.142593,...,0.105556,0.142593,0.068519,0.068519,0.105556,0.105556,0.142593,0.105556,0.068519,0.142593
7,0.053704,0.053704,0.090741,0.090741,0.053704,0.016667,0.016667,0.053704,0.090741,0.090741,...,0.053704,0.090741,0.016667,0.016667,0.053704,0.053704,0.090741,0.053704,0.016667,0.090741
8,0.022593,0.022593,0.059630,0.059630,0.022593,-0.014444,-0.014444,0.022593,0.059630,0.059630,...,0.022593,0.059630,-0.014444,-0.014444,0.022593,0.022593,0.059630,0.022593,-0.014444,0.059630
9,0.032963,0.032963,0.070000,0.070000,0.032963,-0.004074,-0.004074,0.032963,0.070000,0.070000,...,0.032963,0.070000,-0.004074,-0.004074,0.032963,0.032963,0.070000,0.032963,-0.004074,0.070000


In [17]:
item_prediction = predict(ratings_matrix, item_similarity, type='item')

item_prediction.shape

(28, 100)

In [18]:
item_prediction

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
user_id,,,,,,,,,,,,,,,,,,,,,
0d8d07544717e84149df654caae803d0,0.1250,0.112360,0.112360,0.1250,0.106383,0.112360,0.106383,0.1250,0.112360,0.106383,...,0.108696,0.112360,0.104167,0.112360,0.112360,0.112360,0.112360,0.112360,0.112360,0.104167
0ef32090550901ead25cb0ea21c4d36b,0.2500,0.224719,0.224719,0.2500,0.000000,0.224719,0.000000,0.2500,0.224719,0.212766,...,0.217391,0.224719,0.208333,0.224719,0.224719,0.224719,0.224719,0.224719,0.224719,0.208333
18759758c7cf75975cff3fbb1363a73f,0.5125,0.000000,0.460674,0.5125,0.436170,0.000000,0.436170,0.5125,0.000000,0.436170,...,0.445652,0.460674,0.427083,0.000000,0.000000,0.460674,0.460674,0.460674,0.460674,0.427083
1afe8b35c5e568e95bc17e5b5cdbfd1b,0.5625,0.505618,0.000000,0.5625,0.478723,0.505618,0.478723,0.5625,0.505618,0.478723,...,0.489130,0.000000,0.468750,0.505618,0.505618,0.000000,0.000000,0.000000,0.000000,0.468750
2f6af21d14c83a5df6cdcef5e6af0b3e,0.1125,0.101124,0.101124,0.1125,0.095745,0.101124,0.095745,0.1125,0.101124,0.095745,...,0.097826,0.101124,0.093750,0.101124,0.101124,0.101124,0.101124,0.101124,0.101124,0.093750
330f2557be759b71248c49fc4f09992d,0.0625,0.056180,0.056180,0.0625,0.053191,0.056180,0.053191,0.0625,0.056180,0.053191,...,0.054348,0.056180,0.052083,0.056180,0.056180,0.056180,0.056180,0.056180,0.056180,0.052083
3682faa64cbddd0ef719d4a539d6ac51,0.1250,0.112360,0.112360,0.1250,0.106383,0.112360,0.106383,0.1250,0.112360,0.106383,...,0.108696,0.112360,0.104167,0.112360,0.112360,0.112360,0.112360,0.112360,0.112360,0.104167
453e6a280947ac1f82a27b91e5fb89bb,0.0625,0.056180,0.056180,0.0625,0.053191,0.056180,0.053191,0.0625,0.056180,0.053191,...,0.054348,0.056180,0.052083,0.056180,0.056180,0.056180,0.056180,0.056180,0.056180,0.052083
4672eb229c808b792b8ea95f01f19784,0.0250,0.022472,0.022472,0.0250,0.021277,0.022472,0.021277,0.0250,0.022472,0.021277,...,0.021739,0.022472,0.020833,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.020833


In [19]:
# Translate cosine similarity scores to 5-point scale
item_prediction = item_prediction * 5
display(type(item_prediction))

pandas.core.frame.DataFrame

In [22]:
item_prediction.apply(round).astype(int)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
user_id,,,,,,,,,,,,,,,,,,,,,
0d8d07544717e84149df654caae803d0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
0ef32090550901ead25cb0ea21c4d36b,1,1,1,1,0,1,0,1,1,1,...,1,1,1,1,1,1,1,1,1,1
18759758c7cf75975cff3fbb1363a73f,3,0,2,3,2,0,2,3,0,2,...,2,2,2,0,0,2,2,2,2,2
1afe8b35c5e568e95bc17e5b5cdbfd1b,3,3,0,3,2,3,2,3,3,2,...,2,0,2,3,3,0,0,0,0,2
2f6af21d14c83a5df6cdcef5e6af0b3e,1,1,1,1,0,1,0,1,1,0,...,0,1,0,1,1,1,1,1,1,0
330f2557be759b71248c49fc4f09992d,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3682faa64cbddd0ef719d4a539d6ac51,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
453e6a280947ac1f82a27b91e5fb89bb,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4672eb229c808b792b8ea95f01f19784,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
